# Binance Price Data Verification

This notebook verifies the quality and completeness of the Binance minute-by-minute price data.

**Data**: `data/binance_sep_dec28_2025_partial_minute_data.csv`

**Coverage**: Sept 1 - Dec 5, 2025

**Symbols**: BTCUSDT, ETHUSDT, SOLUSDT

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Load Data

In [2]:
# Load the data
df = pd.read_csv('../data/binance_sep_dec28_2025_partial_minute_data.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f"Total rows: {len(df):,}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nColumns: {df.columns.tolist()}")
df.head(10)

Total rows: 77,625
Memory usage: 8.29 MB

Columns: ['symbol', 'timestamp', 'open', 'high', 'low', 'close', 'volume', 'trades']


,symbol,timestamp,open,high,low,close,volume,trades
0,BTCUSDT,2025-09-01 00:00:00,108246.36,108260.00,108210.66,108260.00,15.88924,2717
1,BTCUSDT,2025-09-01 00:01:00,108260.00,108332.35,108259.99,108332.35,12.94030,1309
2,BTCUSDT,2025-09-01 00:02:00,108332.35,108332.35,108256.43,108256.44,25.92896,2136
3,BTCUSDT,2025-09-01 00:03:00,108256.44,108282.43,108229.17,108229.18,18.99223,2344
4,BTCUSDT,2025-09-01 00:04:00,108229.18,108229.18,108100.00,108100.00,12.05048,3790
5,BTCUSDT,2025-09-01 00:05:00,108100.00,108110.46,108060.00,108070.04,45.33450,5961
6,BTCUSDT,2025-09-01 00:06:00,108070.04,108086.17,108015.00,108074.49,26.80768,4855
7,BTCUSDT,2025-09-01 00:07:00,108074.50,108152.44,108043.23,108152.43,17.08691,3203
8,BTCUSDT,2025-09-01 00:08:00,108152.43,108152.43,108034.16,108037.77,75.56924,3241
9,BTCUSDT,2025-09-01 00:09:00,108037.77,108037.78,107983.00,107989.04,88.42085,5989


## 2. Basic Statistics

In [3]:
# Date range
print("=" * 70)
print("DATE RANGE")
print("=" * 70)
print(f"Start: {df['timestamp'].min()}")
print(f"End:   {df['timestamp'].max()}")
print(f"Days:  {(df['timestamp'].max() - df['timestamp'].min()).days} days")

# Per symbol
print("\n" + "=" * 70)
print("PER SYMBOL BREAKDOWN")
print("=" * 70)
print(df.groupby('symbol').agg({
    'timestamp': ['min', 'max', 'count'],
    'volume': ['sum', 'mean'],
    'trades': ['sum', 'mean']
}))

DATE RANGE
Start: 2025-09-01 00:00:00
End:   2025-12-05 20:56:00
Days:  95 days

PER SYMBOL BREAKDOWN
         timestamp                                   volume               \
               min                 max  count           sum         mean   
symbol                                                                     
BTCUSDT 2025-09-01 2025-12-05 20:56:00  25875  3.731818e+05    14.422485   
ETHUSDT 2025-09-01 2025-12-05 20:56:00  25875  9.108531e+06   352.020510   
SOLUSDT 2025-09-01 2025-12-05 20:56:00  25875  7.138797e+07  2758.955382   

           trades               
              sum         mean  
symbol                          
BTCUSDT  75597655  2921.648502  
ETHUSDT  87970659  3399.832232  
SOLUSDT  45025709  1740.124019  


In [ ]:
# Price statistics
print("=" * 70)
print("PRICE STATISTICS")
print("=" * 70)
for symbol in df['symbol'].unique():
    symbol_df = df[df['symbol'] == symbol]
    print(f"\n{symbol}:")
    print(f"  Open:   min=${symbol_df['open'].min():,.2f}, max=${symbol_df['open'].max():,.2f}")
    print(f"  High:   min=${symbol_df['high'].min():,.2f}, max=${symbol_df['high'].max():,.2f}")
    print(f"  Low:    min=${symbol_df['low'].min():,.2f}, max=${symbol_df['low'].max():,.2f}")
    print(f"  Close:  min=${symbol_df['close'].min():,.2f}, max=${symbol_df['close'].max():,.2f}")
    print(f"  Volume: total={symbol_df['volume'].sum():,.2f}, mean={symbol_df['volume'].mean():,.2f}")
    print(f"  Trades: total={symbol_df['trades'].sum():,.0f}, mean={symbol_df['trades'].mean():.1f}")

## 3. Data Quality Checks

In [ ]:
print("=" * 70)
print("DATA QUALITY CHECKS")
print("=" * 70)

# Check for missing values
print("\n1. Missing Values:")
missing = df.isnull().sum()
if missing.sum() == 0:
    print("   ✓ No missing values")
else:
    print(missing[missing > 0])

# Check for duplicates
print("\n2. Duplicate Timestamps:")
duplicates = df.duplicated(subset=['symbol', 'timestamp']).sum()
if duplicates == 0:
    print("   ✓ No duplicate (symbol, timestamp) pairs")
else:
    print(f"   ✗ Found {duplicates} duplicates")

# Check OHLC validity (high >= open, low <= close, etc.)
print("\n3. OHLC Validity:")
invalid_high = (df['high'] < df['open']) | (df['high'] < df['close'])
invalid_low = (df['low'] > df['open']) | (df['low'] > df['close'])
invalid_range = df['high'] < df['low']

total_invalid = invalid_high.sum() + invalid_low.sum() + invalid_range.sum()
if total_invalid == 0:
    print("   ✓ All OHLC relationships valid")
else:
    print(f"   ✗ Invalid high: {invalid_high.sum()}")
    print(f"   ✗ Invalid low: {invalid_low.sum()}")
    print(f"   ✗ Invalid range (high < low): {invalid_range.sum()}")

# Check for zero/negative prices
print("\n4. Price Validity:")
zero_prices = ((df['open'] <= 0) | (df['high'] <= 0) | (df['low'] <= 0) | (df['close'] <= 0)).sum()
if zero_prices == 0:
    print("   ✓ All prices positive")
else:
    print(f"   ✗ Found {zero_prices} rows with zero/negative prices")

# Check for zero volume
print("\n5. Volume/Trades:")
zero_volume = (df['volume'] == 0).sum()
zero_trades = (df['trades'] == 0).sum()
print(f"   Zero volume: {zero_volume} rows ({zero_volume/len(df)*100:.2f}%)")
print(f"   Zero trades: {zero_trades} rows ({zero_trades/len(df)*100:.2f}%)")
if zero_volume == zero_trades:
    print("   ✓ Zero volume matches zero trades (expected)")
else:
    print("   ⚠ Mismatch between zero volume and zero trades")

## 4. Timestamp Completeness

In [ ]:
print("=" * 70)
print("TIMESTAMP COMPLETENESS")
print("=" * 70)

for symbol in df['symbol'].unique():
    symbol_df = df[df['symbol'] == symbol].sort_values('timestamp')
    
    # Expected number of minutes
    start = symbol_df['timestamp'].min()
    end = symbol_df['timestamp'].max()
    expected_minutes = int((end - start).total_seconds() / 60) + 1
    actual_minutes = len(symbol_df)
    
    print(f"\n{symbol}:")
    print(f"  Expected minutes: {expected_minutes:,}")
    print(f"  Actual minutes:   {actual_minutes:,}")
    print(f"  Missing minutes:  {expected_minutes - actual_minutes:,} ({(expected_minutes - actual_minutes)/expected_minutes*100:.2f}%)")
    
    # Check for gaps > 1 minute
    symbol_df = symbol_df.copy()
    symbol_df['time_diff'] = symbol_df['timestamp'].diff()
    gaps = symbol_df[symbol_df['time_diff'] > timedelta(minutes=1)]
    
    if len(gaps) > 0:
        print(f"  Gaps found: {len(gaps)}")
        print(f"  Largest gap: {gaps['time_diff'].max()}")
        print(f"\n  Top 5 largest gaps:")
        top_gaps = gaps.nlargest(5, 'time_diff')[['timestamp', 'time_diff']]
        for idx, row in top_gaps.iterrows():
            print(f"    {row['timestamp']}: {row['time_diff']}")
    else:
        print("  ✓ No gaps > 1 minute")

## 5. Price Movement Analysis

In [ ]:
# Calculate minute-by-minute returns
for symbol in df['symbol'].unique():
    symbol_df = df[df['symbol'] == symbol].sort_values('timestamp').copy()
    symbol_df['returns'] = symbol_df['close'].pct_change() * 100
    df.loc[df['symbol'] == symbol, 'returns'] = symbol_df['returns'].values

print("=" * 70)
print("PRICE MOVEMENT STATISTICS")
print("=" * 70)

for symbol in df['symbol'].unique():
    symbol_df = df[df['symbol'] == symbol]
    returns = symbol_df['returns'].dropna()
    
    print(f"\n{symbol}:")
    print(f"  Mean return:   {returns.mean():.4f}%")
    print(f"  Std dev:       {returns.std():.4f}%")
    print(f"  Min return:    {returns.min():.2f}%")
    print(f"  Max return:    {returns.max():.2f}%")
    print(f"  Median:        {returns.median():.4f}%")
    print(f"  25th pctl:     {returns.quantile(0.25):.4f}%")
    print(f"  75th pctl:     {returns.quantile(0.75):.4f}%")

## 6. Outlier Detection

In [ ]:
print("=" * 70)
print("OUTLIER DETECTION")
print("=" * 70)

for symbol in df['symbol'].unique():
    symbol_df = df[df['symbol'] == symbol]
    returns = symbol_df['returns'].dropna()
    
    # Identify outliers (±3 standard deviations)
    mean = returns.mean()
    std = returns.std()
    outliers = returns[(returns < mean - 3*std) | (returns > mean + 3*std)]
    
    print(f"\n{symbol}:")
    print(f"  Outliers (±3 std): {len(outliers)} ({len(outliers)/len(returns)*100:.3f}%)")
    
    if len(outliers) > 0:
        print(f"  Top 5 positive outliers:")
        top_positive = outliers.nlargest(5)
        for val in top_positive:
            print(f"    {val:.2f}%")
        
        print(f"  Top 5 negative outliers:")
        top_negative = outliers.nsmallest(5)
        for val in top_negative:
            print(f"    {val:.2f}%")

## 7. Visualizations

In [ ]:
# Price trends
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

for idx, symbol in enumerate(sorted(df['symbol'].unique())):
    symbol_df = df[df['symbol'] == symbol].sort_values('timestamp')
    
    axes[idx].plot(symbol_df['timestamp'], symbol_df['close'], linewidth=0.5, alpha=0.8)
    axes[idx].set_title(f'{symbol} - Close Price Over Time', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Date')
    axes[idx].set_ylabel('Price (USD)')
    axes[idx].grid(True, alpha=0.3)
    
    # Add summary stats
    start_price = symbol_df['close'].iloc[0]
    end_price = symbol_df['close'].iloc[-1]
    change_pct = (end_price - start_price) / start_price * 100
    
    axes[idx].text(0.02, 0.95, 
                   f'Start: ${start_price:,.2f}\nEnd: ${end_price:,.2f}\nChange: {change_pct:+.2f}%',
                   transform=axes[idx].transAxes,
                   verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
                   fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Return distributions
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, symbol in enumerate(sorted(df['symbol'].unique())):
    symbol_df = df[df['symbol'] == symbol]
    returns = symbol_df['returns'].dropna()
    
    axes[idx].hist(returns, bins=100, alpha=0.7, edgecolor='black', linewidth=0.5)
    axes[idx].set_title(f'{symbol} - Return Distribution', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Return (%)')
    axes[idx].set_ylabel('Frequency')
    axes[idx].axvline(returns.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {returns.mean():.4f}%')
    axes[idx].axvline(returns.median(), color='green', linestyle='--', linewidth=2, label=f'Median: {returns.median():.4f}%')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Volume analysis
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

for idx, symbol in enumerate(sorted(df['symbol'].unique())):
    symbol_df = df[df['symbol'] == symbol].sort_values('timestamp')
    
    axes[idx].bar(symbol_df['timestamp'], symbol_df['volume'], width=0.0007, alpha=0.6)
    axes[idx].set_title(f'{symbol} - Trading Volume Over Time', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Date')
    axes[idx].set_ylabel('Volume')
    axes[idx].grid(True, alpha=0.3)
    
    # Add summary
    total_vol = symbol_df['volume'].sum()
    avg_vol = symbol_df['volume'].mean()
    
    axes[idx].text(0.02, 0.95,
                   f'Total: {total_vol:,.0f}\nAvg: {avg_vol:,.2f}',
                   transform=axes[idx].transAxes,
                   verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5),
                   fontsize=10)

plt.tight_layout()
plt.show()

## 8. Data Usability Summary

In [ ]:
print("=" * 70)
print("DATA USABILITY ASSESSMENT")
print("=" * 70)

# Calculate overall metrics
total_rows = len(df)
total_symbols = df['symbol'].nunique()
date_range_days = (df['timestamp'].max() - df['timestamp'].min()).days

# Quality checks
has_missing = df.isnull().sum().sum() > 0
has_duplicates = df.duplicated(subset=['symbol', 'timestamp']).sum() > 0
invalid_ohlc = ((df['high'] < df['open']) | (df['high'] < df['close']) | 
                (df['low'] > df['open']) | (df['low'] > df['close']) | 
                (df['high'] < df['low'])).sum() > 0
has_invalid_prices = ((df['open'] <= 0) | (df['high'] <= 0) | 
                      (df['low'] <= 0) | (df['close'] <= 0)).sum() > 0

print(f"\n✓ Dataset Overview:")
print(f"  - {total_rows:,} minute candles")
print(f"  - {total_symbols} symbols (BTCUSDT, ETHUSDT, SOLUSDT)")
print(f"  - {date_range_days} days of data (Sept 1 - Dec 5, 2025)")
print(f"  - File size: 5.50 MB")

print(f"\n✓ Quality Checks:")
if not has_missing:
    print("  ✓ No missing values")
else:
    print("  ✗ Has missing values")
    
if not has_duplicates:
    print("  ✓ No duplicate timestamps")
else:
    print("  ✗ Has duplicate timestamps")
    
if not invalid_ohlc:
    print("  ✓ All OHLC relationships valid")
else:
    print("  ✗ Some invalid OHLC values")
    
if not has_invalid_prices:
    print("  ✓ All prices positive")
else:
    print("  ✗ Some invalid prices")

# Calculate completeness
print(f"\n✓ Data Completeness:")
for symbol in sorted(df['symbol'].unique()):
    symbol_df = df[df['symbol'] == symbol]
    expected = int((symbol_df['timestamp'].max() - symbol_df['timestamp'].min()).total_seconds() / 60) + 1
    actual = len(symbol_df)
    completeness = actual / expected * 100
    print(f"  {symbol}: {completeness:.2f}% complete ({actual:,}/{expected:,} minutes)")

# Overall verdict
print(f"\n" + "=" * 70)
if not (has_missing or has_duplicates or invalid_ohlc or has_invalid_prices):
    print("✓ DATA IS READY TO USE")
    print("  All quality checks passed. No critical issues detected.")
else:
    print("⚠ DATA HAS ISSUES")
    print("  Some quality checks failed. Review issues above.")

print("=" * 70)

## 9. Sample Data Export

Export a small sample for manual inspection

In [ ]:
# Export first 1000 rows for each symbol
sample_dfs = []
for symbol in sorted(df['symbol'].unique()):
    symbol_df = df[df['symbol'] == symbol].sort_values('timestamp').head(1000)
    sample_dfs.append(symbol_df)

sample = pd.concat(sample_dfs)
sample_path = '../data/binance_sample_1000rows_per_symbol.csv'
sample.to_csv(sample_path, index=False)

print(f"✓ Exported sample to: {sample_path}")
print(f"  Rows: {len(sample):,}")
print(f"  Size: {len(sample) * df.memory_usage(deep=True).sum() / len(df) / 1024:.2f} KB")

In [4]:
from datetime import timedelta

print("=" * 70)
print("DETAILED MISSING RANGES")
print("=" * 70)

for symbol in df['symbol'].unique():
    print(f"\n--- {symbol} ---")
    symbol_df = df[df['symbol'] == symbol].sort_values('timestamp')
    symbol_df['time_diff'] = symbol_df['timestamp'].diff()
    
    # Find gaps > 1 minute
    gaps = symbol_df[symbol_df['time_diff'] > timedelta(minutes=1)]
    
    if len(gaps) == 0:
        print("No gaps found.")
    else:
        print(f"Found {len(gaps)} gaps:")
        for idx, row in gaps.iterrows():
            gap_end = row['timestamp']
            gap_duration = row['time_diff']
            # The gap starts 1 minute after the PREVIOUS row
            gap_start = gap_end - gap_duration + timedelta(minutes=1)
            # The gap ends 1 minute before the CURRENT row
            missing_end = gap_end - timedelta(minutes=1)
            
            print(f"  Missing: {gap_start} to {missing_end} | Duration: {gap_duration}")

DETAILED MISSING RANGES

--- BTCUSDT ---
Found 19 gaps:
  Missing: 2025-09-01 21:54:00 to 2025-09-05 23:59:00 | Duration: 4 days 02:07:00
  Missing: 2025-09-06 21:54:00 to 2025-09-10 23:59:00 | Duration: 4 days 02:07:00
  Missing: 2025-09-11 21:25:00 to 2025-09-15 23:59:00 | Duration: 4 days 02:36:00
  Missing: 2025-09-16 21:54:00 to 2025-09-20 23:59:00 | Duration: 4 days 02:07:00
  Missing: 2025-09-21 21:12:00 to 2025-09-25 23:59:00 | Duration: 4 days 02:49:00
  Missing: 2025-09-26 21:08:00 to 2025-09-30 23:59:00 | Duration: 4 days 02:53:00
  Missing: 2025-10-01 21:54:00 to 2025-10-05 23:59:00 | Duration: 4 days 02:07:00
  Missing: 2025-10-06 21:50:00 to 2025-10-10 23:59:00 | Duration: 4 days 02:11:00
  Missing: 2025-10-11 21:20:00 to 2025-10-15 23:59:00 | Duration: 4 days 02:41:00
  Missing: 2025-10-16 21:23:00 to 2025-10-20 23:59:00 | Duration: 4 days 02:38:00
  Missing: 2025-10-21 21:54:00 to 2025-10-25 23:59:00 | Duration: 4 days 02:07:00
  Missing: 2025-10-26 21:23:00 to 2025-10-